1. Data Processing
2. EDA
3. Modeling (Attempts) - avoid overfitting
4. Evaluation 
5. Conclusions (Variable importance)

In [1]:
import pandas as pd
import numpy as np
import logging
from sklearn.preprocessing import OneHotEncoder
from datetime import datetime
from sklearn.preprocessing import StandardScaler , MinMaxScaler
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV

In [2]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()

In [3]:
train_df = pd.read_csv('./train.dsv',sep='|')
train_df.head()

,category,make_id,model_id,price__consumer_gross_euro,mileage,first_registration,condition,modification_time,features_central_locking,features_full_service_history,features_hu_au_neu,features_nonsmoker_vehicle,features_adaptive_cruise_ctl,features_abs
2,EstateCar,1900,9.0,16990.000000,94672.0,200602.0,USED,2011-04-06 00:39:09,True,False,False,False,False,True
3,EstateCar,1900,9.0,9749.990234,176675.0,200606.0,USED,2011-04-06 00:38:39,True,True,False,False,False,True
4,Limousine,1900,8.0,9960.299805,109230.0,200608.0,USED,2011-06-03 19:16:35,True,False,True,False,False,True
5,Limousine,1900,8.0,21899.570312,22275.0,201002.0,USED,2011-06-03 19:16:40,True,False,True,False,False,True
6,EstateCar,1900,9.0,7700.000000,244000.0,200408.0,USED,2011-06-03 19:22:40,True,True,False,False,False,True


## EDA

In [4]:
train_df.shape

(12876, 14)

In [5]:
train_df.dtypes

category                          object
make_id                            int64
model_id                         float64
price__consumer_gross_euro       float64
mileage                          float64
first_registration               float64
condition                         object
modification_time                 object
features_central_locking            bool
features_full_service_history       bool
features_hu_au_neu                  bool
features_nonsmoker_vehicle          bool
features_adaptive_cruise_ctl        bool
features_abs                        bool
dtype: object

In [6]:
for col in ['category','make_id','model_id','condition']:
    logger.info(f"col {col} has unique values : '{train_df[col].unique()}, \
        with count = {len(train_df[col].unique())}")

INFO:root:col category has unique values : '['EstateCar' 'Limousine' 'SportsCar' 'Cabrio' 'OffRoad' 'SmallCar'
 'OtherCar' 'Van'],         with count = 8
INFO:root:col make_id has unique values : '[ 1900 25100],         with count = 2
INFO:root:col model_id has unique values : '[ 9.  8. 10. 23. 32. 11.  1. 12. 31. 15. 16.  3. 33. 28. 27.  4. 13. 29.
 19. 20. 30.  2.  5. 22. 21.  7. 14. 24.  6. 25. 34. 37. 39. 36. 17. 35.
 40. 26. 41. 38. 42. 18. nan],         with count = 43
INFO:root:col condition has unique values : '['USED' 'NEW'],         with count = 2


In [7]:
idx = np.isnan(train_df['first_registration'])
train_df.loc[idx,].head()

,category,make_id,model_id,price__consumer_gross_euro,mileage,first_registration,condition,modification_time,features_central_locking,features_full_service_history,features_hu_au_neu,features_nonsmoker_vehicle,features_adaptive_cruise_ctl,features_abs
50,EstateCar,1900,8.0,19999.000000,0.0,NaN,NEW,2009-10-05 13:58:47,True,False,False,False,False,True
53,SportsCar,1900,31.0,29290.750000,0.0,NaN,NEW,2009-10-05 13:58:37,True,False,False,False,False,True
114,Cabrio,1900,9.0,33888.820312,0.0,NaN,NEW,2009-10-28 13:15:57,True,False,False,False,False,True
115,Limousine,1900,11.0,55889.539062,0.0,NaN,NEW,2009-10-28 13:15:57,True,False,False,False,False,True
117,Limousine,1900,8.0,20448.960938,0.0,NaN,NEW,2009-10-28 13:15:58,True,False,False,False,False,True


In [8]:
# Data Understanding 
# what is modification time ?

## Data Preprocessing

In [9]:
# parse first registration yyyy-mm.0
# s = str(int(200602.0))
# datetime.strptime(s,'%Y%m')
def preprocess(train_df):
    train_df['first_registration_date'] = pd.to_datetime(train_df['first_registration'].\
                apply(lambda x: np.datetime64('NaT') if np.isnan(x) else datetime.strptime(str(int(x)),'%Y%m')))
    train_df['modification_time'] = pd.to_datetime(train_df['modification_time'])
    for col in train_df.columns:
        if train_df[col].dtype==bool:
            train_df[col] = train_df[col].astype(int)
    return train_df

In [10]:
train_df_pp = preprocess(train_df)
train_df_pp.head()

,category,make_id,model_id,price__consumer_gross_euro,mileage,first_registration,condition,modification_time,features_central_locking,features_full_service_history,features_hu_au_neu,features_nonsmoker_vehicle,features_adaptive_cruise_ctl,features_abs,first_registration_date
2,EstateCar,1900,9.0,16990.000000,94672.0,200602.0,USED,2011-04-06 00:39:09,1,0,0,0,0,1,2006-02-01
3,EstateCar,1900,9.0,9749.990234,176675.0,200606.0,USED,2011-04-06 00:38:39,1,1,0,0,0,1,2006-06-01
4,Limousine,1900,8.0,9960.299805,109230.0,200608.0,USED,2011-06-03 19:16:35,1,0,1,0,0,1,2006-08-01
5,Limousine,1900,8.0,21899.570312,22275.0,201002.0,USED,2011-06-03 19:16:40,1,0,1,0,0,1,2010-02-01
6,EstateCar,1900,9.0,7700.000000,244000.0,200408.0,USED,2011-06-03 19:22:40,1,1,0,0,0,1,2004-08-01


## Feature Generation

In [11]:
train_df_pp.dtypes

category                                 object
make_id                                   int64
model_id                                float64
price__consumer_gross_euro              float64
mileage                                 float64
first_registration                      float64
condition                                object
modification_time                datetime64[ns]
features_central_locking                  int64
features_full_service_history             int64
features_hu_au_neu                        int64
features_nonsmoker_vehicle                int64
features_adaptive_cruise_ctl              int64
features_abs                              int64
first_registration_date          datetime64[ns]
dtype: object

In [12]:
def to_datetime(date):
    """
    Converts a numpy datetime64 object to a python datetime object 
    Input:
      date - a np.datetime64 object
    Output:
      DATE - a python datetime object
    """
    timestamp = ((date - np.datetime64('1970-01-01T00:00:00'))
                 / np.timedelta64(1, 's'))
    return datetime.utcfromtimestamp(timestamp)

In [13]:
def gen_features(train_df):
    train_df['months_since_first_registration'] = train_df['first_registration_date']\
                    .apply(lambda x:np.NaN if pd.isna(x) else (datetime.today()-x).days/30)
    train_df['months_since_modification_time'] = train_df['modification_time']\
                    .apply(lambda x:np.NaN if pd.isna(x) else (datetime.today()-x).days/30)
    
    # delete unneeded cols
    
    train_df.drop(columns = ['first_registration','first_registration_date','modification_time'],inplace=True)
    categories_cols = ['category','make_id','model_id','condition']
    #enc = OneHotEncoder(categories=categories)
    train_df_dummy = pd.get_dummies(data=train_df,columns=categories_cols,prefix=categories_cols,drop_first=True)
    #train_df_dummy.drop(columns=categories,axis=1)
    return train_df_dummy

In [14]:
train_df_features = gen_features(train_df_pp)
train_df_features.head()

,price__consumer_gross_euro,mileage,features_central_locking,features_full_service_history,features_hu_au_neu,features_nonsmoker_vehicle,features_adaptive_cruise_ctl,features_abs,months_since_first_registration,months_since_modification_time,...,model_id_34.0,model_id_35.0,model_id_36.0,model_id_37.0,model_id_38.0,model_id_39.0,model_id_40.0,model_id_41.0,model_id_42.0,condition_USED
2,16990.000000,94672.0,1,0,0,0,0,1,169.833333,106.833333,...,0,0,0,0,0,0,0,0,0,1
3,9749.990234,176675.0,1,1,0,0,0,1,165.833333,106.833333,...,0,0,0,0,0,0,0,0,0,1
4,9960.299805,109230.0,1,0,1,0,0,1,163.800000,104.866667,...,0,0,0,0,0,0,0,0,0,1
5,21899.570312,22275.0,1,0,1,0,0,1,121.133333,104.866667,...,0,0,0,0,0,0,0,0,0,1
6,7700.000000,244000.0,1,1,0,0,0,1,188.133333,104.866667,...,0,0,0,0,0,0,0,0,0,1


In [15]:
train_df_features.dtypes

price__consumer_gross_euro         float64
mileage                            float64
features_central_locking             int64
features_full_service_history        int64
features_hu_au_neu                   int64
features_nonsmoker_vehicle           int64
features_adaptive_cruise_ctl         int64
features_abs                         int64
months_since_first_registration    float64
months_since_modification_time     float64
category_EstateCar                   uint8
category_Limousine                   uint8
category_OffRoad                     uint8
category_OtherCar                    uint8
category_SmallCar                    uint8
category_SportsCar                   uint8
category_Van                         uint8
make_id_25100                        uint8
model_id_2.0                         uint8
model_id_3.0                         uint8
model_id_4.0                         uint8
model_id_5.0                         uint8
model_id_6.0                         uint8
model_id_7.

In [16]:
train_df_features.to_csv('train_matrix.csv',sep=',',index=False)

## Modeling
### To attempt:
1. NN
2. SVR
3. Boosting
4. Baseline , regression model

### Model (1) MLP:

In [17]:
train_df_features.head()

,price__consumer_gross_euro,mileage,features_central_locking,features_full_service_history,features_hu_au_neu,features_nonsmoker_vehicle,features_adaptive_cruise_ctl,features_abs,months_since_first_registration,months_since_modification_time,...,model_id_34.0,model_id_35.0,model_id_36.0,model_id_37.0,model_id_38.0,model_id_39.0,model_id_40.0,model_id_41.0,model_id_42.0,condition_USED
2,16990.000000,94672.0,1,0,0,0,0,1,169.833333,106.833333,...,0,0,0,0,0,0,0,0,0,1
3,9749.990234,176675.0,1,1,0,0,0,1,165.833333,106.833333,...,0,0,0,0,0,0,0,0,0,1
4,9960.299805,109230.0,1,0,1,0,0,1,163.800000,104.866667,...,0,0,0,0,0,0,0,0,0,1
5,21899.570312,22275.0,1,0,1,0,0,1,121.133333,104.866667,...,0,0,0,0,0,0,0,0,0,1
6,7700.000000,244000.0,1,1,0,0,0,1,188.133333,104.866667,...,0,0,0,0,0,0,0,0,0,1


### Features, Scaling / Normalizing
https://towardsdatascience.com/scale-standardize-or-normalize-with-scikit-learn-6ccc7d176a02

In [18]:
def scale_normalize_features(train_df):    
    # TODO experiment other scalars based on better EDA for cols distributions
    features_to_scale = ['mileage','months_since_first_registration','months_since_modification_time']
    scaler = MinMaxScaler().fit(train_df[features_to_scale])
    mtx_scaled_features = pd.DataFrame(data=scaler.transform(train_df[features_to_scale])\
                                       ,columns=[f +'_scaled' for f in features_to_scale])
    train_df_dropped = train_df.drop(columns = features_to_scale)
    train_df_scaled = pd.concat([train_df_dropped,mtx_scaled_features],axis=1)
    return train_df_scaled

In [19]:
train_df_features_scaled = scale_normalize_features(train_df_features)
train_df_features_scaled.head()

,price__consumer_gross_euro,features_central_locking,features_full_service_history,features_hu_au_neu,features_nonsmoker_vehicle,features_adaptive_cruise_ctl,features_abs,category_EstateCar,category_Limousine,category_OffRoad,...,model_id_37.0,model_id_38.0,model_id_39.0,model_id_40.0,model_id_41.0,model_id_42.0,condition_USED,mileage_scaled,months_since_first_registration_scaled,months_since_modification_time_scaled
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.039121,0.149023,0.704698
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.073006,0.143478,0.704698
2,16990.000000,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.045136,0.140659,0.674239
3,9749.990234,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.009205,0.081512,0.674239
4,9960.299805,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.100826,0.174391,0.674239


In [20]:
#https://scikit-learn.org/stable/auto_examples/plot_partial_dependence_visualization_api.html#sphx-glr-auto-examples-plot-partial-dependence-visualization-api-py

In [21]:
y_colname = 'price__consumer_gross_euro'
y = train_df_features_scaled[y_colname]
X = train_df_features_scaled.drop(columns=y_colname)
X.columns

Index(['features_central_locking', 'features_full_service_history',
       'features_hu_au_neu', 'features_nonsmoker_vehicle',
       'features_adaptive_cruise_ctl', 'features_abs', 'category_EstateCar',
       'category_Limousine', 'category_OffRoad', 'category_OtherCar',
       'category_SmallCar', 'category_SportsCar', 'category_Van',
       'make_id_25100', 'model_id_2.0', 'model_id_3.0', 'model_id_4.0',
       'model_id_5.0', 'model_id_6.0', 'model_id_7.0', 'model_id_8.0',
       'model_id_9.0', 'model_id_10.0', 'model_id_11.0', 'model_id_12.0',
       'model_id_13.0', 'model_id_14.0', 'model_id_15.0', 'model_id_16.0',
       'model_id_17.0', 'model_id_18.0', 'model_id_19.0', 'model_id_20.0',
       'model_id_21.0', 'model_id_22.0', 'model_id_23.0', 'model_id_24.0',
       'model_id_25.0', 'model_id_26.0', 'model_id_27.0', 'model_id_28.0',
       'model_id_29.0', 'model_id_30.0', 'model_id_31.0', 'model_id_32.0',
       'model_id_33.0', 'model_id_34.0', 'model_id_35.0', 'model_id_

In [22]:
X.shape

(17423, 59)

In [23]:
y.shape

(17423,)

In [24]:
# Drop any row having NA, think of better imputing approach
x_na_idx = X.isnull().any(1)
X_non_na = X.loc[~x_na_idx,]
y_non_na = y.loc[~x_na_idx,]

In [25]:
mlp = MLPRegressor(hidden_layer_sizes=(1000),tol=1e-2, max_iter=10000, random_state=0,verbose=True\
                   ,learning_rate_init=5,learning_rate='adaptive')
mlp_fit = mlp.fit(X_non_na,y_non_na)
mlp_fit

Iteration 1, loss = 489540252.55816853
Iteration 2, loss = 437197451.85388541
Iteration 3, loss = 433243625.13811183
Iteration 4, loss = 429259633.66251320
Iteration 5, loss = 425284263.02567321
Iteration 6, loss = 421324307.24648905
Iteration 7, loss = 417370313.16285264
Iteration 8, loss = 413473167.14989877
Iteration 9, loss = 409629266.83263999
Iteration 10, loss = 405850884.77855861
Iteration 11, loss = 402130018.42512560
Iteration 12, loss = 398466633.81605560
Iteration 13, loss = 394820922.71113843
Iteration 14, loss = 391204996.11520010
Iteration 15, loss = 387656029.54353064
Iteration 16, loss = 384168011.73014790
Iteration 17, loss = 380694612.40366364
Iteration 18, loss = 377289018.15077025
Iteration 19, loss = 373928187.65195805
Iteration 20, loss = 370612369.56433785
Iteration 21, loss = 367326440.02411580
Iteration 22, loss = 364055490.66142732
Iteration 23, loss = 360862482.19380999
Iteration 24, loss = 357670655.22201365
Iteration 25, loss = 354547897.03394234
Iteration

Iteration 204, loss = 130163092.55206101
Iteration 205, loss = 130435597.04944666
Iteration 206, loss = 131404848.41833431
Iteration 207, loss = 130525377.11044137
Iteration 208, loss = 130689844.62460251
Iteration 209, loss = 130519681.22303653
Iteration 210, loss = 130285659.19540399
Iteration 211, loss = 129847747.46653594
Iteration 212, loss = 131656388.34574702
Iteration 213, loss = 130068722.31860529
Iteration 214, loss = 130944723.65651901
Iteration 215, loss = 130587963.46882194
Iteration 216, loss = 130249124.43695363
Iteration 217, loss = 130990849.79206768
Iteration 218, loss = 130140384.50224751
Iteration 219, loss = 130453433.14719935
Iteration 220, loss = 129793098.55245546
Iteration 221, loss = 136553140.92770353
Iteration 222, loss = 130313687.15790963
Iteration 223, loss = 130171198.61153808
Iteration 224, loss = 129905829.47050533
Iteration 225, loss = 130350550.85328659
Iteration 226, loss = 129816508.39065571
Iteration 227, loss = 130922915.54344873
Iteration 228, l

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=1000, learning_rate='adaptive',
             learning_rate_init=5, max_fun=15000, max_iter=10000, momentum=0.9,
             n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
             random_state=0, shuffle=True, solver='adam', tol=0.01,
             validation_fraction=0.1, verbose=True, warm_start=False)

### Results Log
MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=100, learning_rate='adaptive',
             learning_rate_init=1, max_fun=15000, max_iter=10000, momentum=0.9,
             n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
             random_state=0, shuffle=True, solver='adam', tol=0.01,
             validation_fraction=0.1, verbose=True, warm_start=False) 
             
             
 Iteration 158, loss = 84140825.32805520

""""""""""

Iteration 167, loss = 69278334.91476229


Training loss did not improve more than tol=0.010000 for 10 consecutive epochs. Stopping.
MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=500, learning_rate='adaptive',
             learning_rate_init=1, max_fun=15000, max_iter=10000, momentum=0.9,
             n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
             random_state=0, shuffle=True, solver='adam', tol=0.01,
             validation_fraction=0.1, verbose=True, warm_start=False)

"""""""""""""""""""""2

In [26]:
# use gridcvsearch

In [36]:
param_grid = {'activation':['logistic','tanh', 'relu'],\
             'hidden_layer_sizes':[(200,200,200)],\
             'learning_rate_init':[1,10],\
             'tol' : [0.001],\
             'alpha':list(10.0 ** -np.arange(1, 2))}

mlp = MLPRegressor()
grid_search_cv_fit = GridSearchCV(estimator=mlp,param_grid=param_grid,verbose=True\
                                  ,scoring='neg_root_mean_squared_error'\
                                  ,n_jobs=4)\
                .fit(X_non_na,y_non_na)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  30 out of  30 | elapsed:  1.8min finished


In [37]:
grid_search_cv_fit.best_estimator_

MLPRegressor(activation='relu', alpha=0.1, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(200, 200, 200), learning_rate='constant',
             learning_rate_init=1, max_fun=15000, max_iter=200, momentum=0.9,
             n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
             random_state=None, shuffle=True, solver='adam', tol=0.001,
             validation_fraction=0.1, verbose=False, warm_start=False)

In [39]:
nrmse = grid_search_cv_fit.best_score_

In [41]:
-nrmse/(max(y_non_na)-min(y_non_na))

0.06101495519046771

## Grid CV results summary

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.

[Parallel(n_jobs=4)]: Done  40 out of  40 | elapsed:  3.1min finished

In [30]:
# cross validation
# over under estimating prices
# https://scikit-learn.org/stable/modules/neural_networks_supervised.html#tips-on-practical-use
# https://machinelearningmastery.com/how-to-configure-the-number-of-layers-and-nodes-in-a-neural-network/
#https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

In [ ]:
# report r2 and nrmse , mat5omsh :D
